In [13]:
import pandas as pd
import numpy as np

In [14]:
A = np.array([[1, 2, 3, 2, 3, 2, 1], [4, 5, 6, 6, 3, 1, 2], [7, 8, 9, 3, 1, 2, 3]])

df = pd.DataFrame(A, columns=['A_x', 'A_y', 'B_x', 'B_y', 'C_x', 'C_y', 'label'])
print(df)

   A_x  A_y  B_x  B_y  C_x  C_y  label
0    1    2    3    2    3    2      1
1    4    5    6    6    3    1      2
2    7    8    9    3    1    2      3


In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Dữ liệu mẫu
data = {
    'A_x': [1, 4, 7],
    'A_y': [2, 5, 8],
    'B_x': [3, 6, 9],
    'B_y': [2, 6, 3],
    'C_x': [3, 3, 1],
    'C_y': [2, 1, 2],
    'label': [1, 2, 3]
}

# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(data)

# Chia dữ liệu thành features (X) và label (y)
X = df.drop('label', axis=1)
y = df['label']

# Nhóm các cặp feature
feature_groups = [
    ['A_x', 'A_y'],
    ['B_x', 'B_y'],
    ['C_x', 'C_y']
]

# Xây dựng RFE với các nhóm feature
class GroupedRFE:
    def __init__(self, estimator, n_groups_to_select):
        self.estimator = estimator
        self.n_groups_to_select = n_groups_to_select
        self.support_ = None

    def fit(self, X, y):
        # Ban đầu đánh dấu tất cả các nhóm là có thể lựa chọn
        n_groups = len(feature_groups)
        group_support_ = np.ones(n_groups, dtype=bool)

        while np.sum(group_support_) > self.n_groups_to_select:
            # Lấy các cột hiện tại tương ứng với các nhóm feature còn được chọn
            current_features = np.hstack([feature_groups[i] for i in range(n_groups) if group_support_[i]])
            X_subset = X[current_features]
            
            # Huấn luyện mô hình
            self.estimator.fit(X_subset, y)
            
            # Tính toán độ quan trọng của từng nhóm
            group_importances = []
            for i, group in enumerate(feature_groups):
                if group_support_[i]:
                    X_group = X[group]
                    group_importances.append(self.estimator.feature_importances_[X.columns.get_indexer(group)].mean())
                else:
                    group_importances.append(np.inf)  # Không xét nhóm đã bị loại

            # Loại bỏ nhóm ít quan trọng nhất
            group_support_[np.argmin(group_importances)] = False

        # Lưu lại các nhóm được chọn
        self.support_ = np.hstack([group_support_[i] for i in range(n_groups)])

    def transform(self, X):
        # Chỉ giữ lại các nhóm feature đã được chọn
        selected_features = np.hstack([feature_groups[i] for i in range(len(feature_groups)) if self.support_[i]])
        return X[selected_features]

# Chia dữ liệu thành train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình (ở đây dùng RandomForest)
model = RandomForestClassifier()

# Khởi tạo GroupedRFE và lựa chọn 2 nhóm feature
grouped_rfe = GroupedRFE(model, n_groups_to_select=2)
grouped_rfe.fit(X_train, y_train)

# Kiểm tra các nhóm feature được chọn
selected_groups = [group for group, support in zip(feature_groups, grouped_rfe.support_) if support]
print("Selected feature groups:", selected_groups)

# Biến đổi dữ liệu để chỉ giữ các feature đã chọn
X_selected_train = grouped_rfe.transform(X_train)
X_selected_test = grouped_rfe.transform(X_test)

# Bạn có thể tiếp tục huấn luyện mô hình với các feature đã được chọn
model.fit(X_selected_train, y_train)


Selected feature groups: [['A_x', 'A_y'], ['B_x', 'B_y']]


RandomForestClassifier()

In [18]:
print("X_train:")
print(X_train)

X_train:
   A_x  A_y  B_x  B_y  C_x  C_y
1    4    5    6    6    3    1
2    7    8    9    3    1    2
